In [3]:
from bs4 import BeautifulSoup
import httpx
from typing import Callable
from functools import reduce
from rich import print
PTT_OVER_18_HEADER = {"cookie": "over18=1"}

In [4]:

def page_to_simple_dict(html_str: str, func_list: list[Callable] = None) -> dict:
    """
    format like
    '作者': 'ReiKuromiya (ReiKuromiya)',
    '標題': '[正妹] 周子瑜',
    '時間': 'Sun Jan  1 00:26:06 2023',
    'Year': '2023',
    'Month': 'Jan',
    'Day': '',
    'Week': 'Sun',
    'Time': '00:26:06',
    # 'Body': [(..., ...),  ...]}
    """

    def to_detail_date(date_str: str) -> dict:
        detail_date = date_str.split(" ")

        if "" in detail_date:
            detail_date.remove("")
        return {
            "Year": detail_date[-1],
            "Month": detail_date[1],
            "Day": detail_date[2],
            "Week": detail_date[0],
            "Time": detail_date[-2],
        }

    # header = {"cookie": "over18=1"}
    # result = httpx.get(url=url, headers=header)

    soup = BeautifulSoup(html_str, "html.parser")

    # get main data
    body_data = soup.find("div", class_="bbs-screen bbs-content", id="main-content")

    # get header data
    header_data = body_data.find_all("div", class_="article-metaline")

    tab_list = [
        str(line.find("span", class_="article-meta-tag").string)
        for line in header_data
    ]

    value_list = [
        str(line.find("span", class_="article-meta-value").string)
        for line in header_data
    ]

    header_dict = dict(zip(tab_list, value_list))

    if "時間" not in header_dict:
        # get text body #https://www.ptt.cc/bbs/Beauty/M.1690589266.A.166.html
        process_text = body_data.contents[2]
        process_text = process_text.split("\n")[:2]
        process_text = [item.split(":") for item in process_text]
        process_result_dict = {item[0]: item[1] for item in process_text}
        header_dict |= process_result_dict

    

    page_data = (
        header_dict
        | to_detail_date(header_dict["時間"])
        
    )

    if func_list is not None:
        addition_dict_list = reduce(
            lambda a, b: a | b, [func(soup) for func in func_list]
        )
        page_data |= addition_dict_list

    return page_data

In [5]:
def get_images_from_page(soup:BeautifulSoup)->dict:
    body_data = soup.find("div", class_="bbs-screen bbs-content", id="main-content")
    # image src
    images_catch_link = body_data.find_all("div", class_="richcontent")

    images_catch_link = [
        item.get("src")
        for image in images_catch_link
        if (item := image.find("img"))
    ]

    # image link lists
    image_link = body_data.find_all("a")
    image_link = [
        link_str
        for link in image_link
        if any(
            substring in (link_str := str(link.string))
            for substring in [".png", ".jpg", "jpeg", ".gif"]
        )
    ]
    return  {
            "image_catch_list": images_catch_link,
            "image_link": image_link,
        }

In [6]:
# test_response = httpx.get("https://www.ptt.cc/bbs/Beauty/M.1692517466.A.D58.html" , headers=PTT_OVER_18_HEADER)
# test_response = httpx.get("https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5B5.html" , headers=PTT_OVER_18_HEADER)
# test_response = httpx.get("https://www.ptt.cc/bbs/Beauty/M.1678508772.A.163.html" , headers=PTT_OVER_18_HEADER)
test_response = httpx.get("https://www.ptt.cc/bbs/Beauty/M.1672510614.A.C61.html" , headers=PTT_OVER_18_HEADER)
test_response.text

'<!DOCTYPE html>\n<html>\n\t<head>\n\t\t<meta charset="utf-8">\n\t\t\n\n<meta name="viewport" content="width=device-width, initial-scale=1">\n\n<title>[正妹] 六兔興旺 - 看板 Beauty - 批踢踢實業坊</title>\n<meta name="robots" content="all">\n<meta name="keywords" content="Ptt BBS 批踢踢">\n<meta name="description" content="https://i.imgur.com/XaKrUlM.jpg 歳末新春SUPERキャンペーン\nhttps://i.imgur.com/uXh36LW.png miru\nhttps://i.imgur.com/IdWjvsc.png\nhttps://i.imgur.com/0gAyDJP.png\nhttps://i.imgur.com/YOO1vKw.png\n">\n<meta property="og:site_name" content="Ptt 批踢踢實業坊">\n<meta property="og:title" content="[正妹] 六兔興旺">\n<meta property="og:description" content="https://i.imgur.com/XaKrUlM.jpg 歳末新春SUPERキャンペーン\nhttps://i.imgur.com/uXh36LW.png miru\nhttps://i.imgur.com/IdWjvsc.png\nhttps://i.imgur.com/0gAyDJP.png\nhttps://i.imgur.com/YOO1vKw.png\n">\n<link rel="canonical" href="https://www.ptt.cc/bbs/Beauty/M.1672510614.A.C61.html">\n\n<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.cs

In [46]:
def get_body_content(soup:BeautifulSoup)->dict:
    body_data = soup.find("div", class_="bbs-screen bbs-content", id="main-content")
    
    body_text = body_data.text.strip()
    
    index_end = body_text.find("※ 發信站")
    # must have ※ 發信站
    body_text = body_text[:index_end]
    print(body_text)
    
    return {"is_can_use" : bool(index_end != -1) ,  "body_content":body_text }

In [47]:
page_dict = page_to_simple_dict(test_response.text,func_list=[get_body_content , get_images_from_page] ) # , func_list=[get_images_from_page]
print(page_dict)
# print(f"image_catch_list size : {len(page_dict['image_catch_list'])}")
# print(f"image_link size : {len(page_dict['image_link'])}")

作者jay228 (Jeer)看板Beauty標題[正妹] 六兔興旺時間Sun Jan  1 02:16:52 2023
https://i.imgur.com/XaKrUlM.jpg 歳末新春SUPERキャンペーン

https://i.imgur.com/uXh36LW.png miru
https://i.imgur.com/IdWjvsc.png
https://i.imgur.com/0gAyDJP.png
https://i.imgur.com/YOO1vKw.png
https://i.imgur.com/sIFeFNb.png
https://i.imgur.com/s21w4ut.png

https://i.imgur.com/LR2UOd9.png 小野六花
https://i.imgur.com/LhlGvVh.png

https://i.imgur.com/s8RHG0a.png 加美杏奈
https://i.imgur.com/zGcqYVz.png

https://i.imgur.com/m8bZ5kD.png 伊藤舞雪
https://i.imgur.com/Mtx8H3Q.png

https://i.imgur.com/o5uM54v.png 山岸逢花
https://i.imgur.com/zUwGWOn.png

https://i.imgur.com/QusrQ0P.png 竹内有紀
https://i.imgur.com/qZjBFAw.png

--

{
    '作者': 'jay228 (Jeer)',
    '標題': '[正妹] 六兔興旺',
    '時間': 'Sun Jan  1 02:16:52 2023',
    'Year': '2023',
    'Month': 'Jan',
    'Day': '1',
    'Week': 'Sun',
    'Time': '02:16:52',
    'is_can_use': True,
    'body_content': '作者jay228 (Jeer)看板Beauty標題[正妹] 六兔興旺時間Sun Jan  1 02:16:52 
2023\nhttps://i.imgur.com/XaKrUlM.jpg 歳末新春SUPERキャンペーン\n\nhttps://i.imgur.com/uXh36LW.png 
miru\nhttps://i.imgur.com/IdWjvsc.png\nhttps://i.imgur.com/0gAyDJP.png\nhttps://i.imgur.com/YOO1vKw.png\nhttps://i.
imgur.com/sIFeFNb.png\nhttps://i.imgur.com/s21w4ut.png\n\nhttps://i.imgur.com/LR2UOd9.png 
小野六花\nhttps://i.imgur.com/LhlGvVh.png\n\nhttps://i.imgur.com/s8RHG0a.png 
加美杏奈\nhttps://i.imgur.com/zGcqYVz.png\n\nhttps://i.imgur.com/m8bZ5kD.png 
伊藤舞雪\nhttps://i.imgur.com/Mtx8H3Q.png\n\nhttps://i.imgur.com/o5uM54v.png 
山岸逢花\nhttps://i.imgur.com/zUwGWOn.png\n\nhttps://i.imgur.com/QusrQ0P.png 
竹内有紀\nhttps://i.imgur.com/qZjBFAw.png\n\n--\n',
    'image_catch_list': [
        'https://cache.ptt.cc/c/https/i.imgur.com/XaKrUlMl.jpg?e=1709353478&s=y_QuMlHrUNVby2DgycvRvA',
        'https://cache.ptt.cc/c/https/i.imgur.com/uXh36LWl.png?e=1709334763&s=Rfh_nYLF8uHnEAh06eo3-Q',
        'https://cache.ptt.cc/c/https/i.imgur.com/IdWjvscl.png?e=1709339632&s=bPda5qqJxJyOScAy1CKRkw',
        'https://cache.ptt.cc/c/https/i.imgur.com/0gAyDJPl.png?e=1709323707&s=XMxHKsOPutJzEZYtTpPcJg',
        'https://cache.ptt.cc/c/https/i.imgur.com/YOO1vKwl.png?e=1709343696&s=KYYkTcHByMDpT-h8LmL-uw',
        'https://cache.ptt.cc/c/https/i.imgur.com/sIFeFNbl.png?e=1709314221&s=tcna6PUqXBGZn_DytLgh6g',
        'https://cache.ptt.cc/c/https/i.imgur.com/s21w4utl.png?e=1709323382&s=qghMKnqR_QNLkjJTctPCRg',
        'https://cache.ptt.cc/c/https/i.imgur.com/LR2UOd9l.png?e=1709349351&s=RZcjkO_mp_orQSHF4pQR9A',
        'https://cache.ptt.cc/c/https/i.imgur.com/LhlGvVl.png?e=1709312435&s=LXEyRT3QR--biQCjj7ZPUg',
        'https://cache.ptt.cc/c/https/i.imgur.com/s8RHG0al.png?e=1709318177&s=VDVIxVnFuacMxM-MOIU9xg',
        'https://cache.ptt.cc/c/https/i.imgur.com/zGcqYVzl.png?e=1709318298&s=2BObVtOASCSzbEGNOuekTA',
        'https://cache.ptt.cc/c/https/i.imgur.com/m8bZ5kDl.png?e=1709352843&s=gbdUv7T9OtvINnSmKhJCgw',
        'https://cache.ptt.cc/c/https/i.imgur.com/Mtx8H3Ql.png?e=1709339819&s=rq3iMEqYr6kqvh6GcnpDFQ',
        'https://cache.ptt.cc/c/https/i.imgur.com/o5uM54vl.png?e=1709362355&s=tC3JVvRbXaG9ZKoEE-tE4A',
        'https://cache.ptt.cc/c/https/i.imgur.com/zUwGWOnl.png?e=1709315137&s=hiExw2gfyFwy-AYXKgGylA',
        'https://cache.ptt.cc/c/https/i.imgur.com/QusrQ0Pl.png?e=1709341498&s=snV5mfL6NtXPheAb_0pROQ',
        'https://cache.ptt.cc/c/https/i.imgur.com/qZjBFAwl.png?e=1709341347&s=aVn5iT7I9PTVtYOHEtE9ng',
        'https://cache.ptt.cc/c/https/i.imgur.com/nSLPzpUl.jpg?e=1709370292&s=lsI4BCQgX3pJfemTpLS7HA',
        'https://cache.ptt.cc/c/https/i.imgur.com/z3TWVzYl.jpg?e=1709348737&s=zTynid-RwJrouzug9hdZSg',
        'https://cache.ptt.cc/c/https/i.imgur.com/1eEWrRDl.jpg?e=1709330930&s=MIle52kqzi5p0JoJW0OLDw',
        'https://cache.ptt.cc/c/https/i.imgur.com/2velvFRl.jpg?e=1709345877&s=ee0fFKAM1EZ3Gg_mtA_2pA',
        'https://cache.ptt.cc/c/https/i.imgur.com/9x4j50tl.jpg?e=1709335182&s=80XbDiBuMPhAFgzO5iiD0Q',
        'https://cache.ptt.cc/c/https/i.imgur.com/g4Ix06Nl.jpg?e=1709348414&s=ZJD_A1fCSlcJINbTHZDiGw',
        'https://cache.ptt.cc/c/https/i.imgur.com/C6mnIWAl.jpg?e=1709330845&s=M6paTp7QCr8IaLxkt5w-Aw'
    ],
    'image_link': [
        'https://i.imgur.com/XaKrUlM.jpg',
        'https://i.imgur.com/uXh36LW.png',
        'https://i.imgur.com/IdWjvsc.png',
        'https://i.imgur.com/0gAyDJP.png',
        'https://i.imgur.com/YOO1vKw.png',
        'https://i.imgur.com/sIFeFNb.png',
        'https://i.imgur.com/s21w4ut.png',
        'https://i.imgur.com/LR2UOd9.png',
        'https://i.imgur.com/LhlGvVh.png',
        'https://i.imgur.com/s8RHG0a.png',
        'https://i.imgur.com/zGcqYVz.png',
        'https://i.imgur.com/m8bZ5kD.png',
        'https://i.imgur.com/